In [27]:
import pandas as pd
import traintools
from trainconstants import *
from collections import defaultdict
import networkx as nx
from typing import List, Tuple

In [28]:
def create_network_schedule(df: pd.DataFrame) -> pd.DataFrame:
    """Creates a DataFrame which can be turned into a networkx graph.
    """

    network_schedule = df[['first_class', 'second_class']].copy()
    network_schedule['start'] = list(zip(df.start, df.departure_time))
    network_schedule['end'] = list(zip(df.end, df.arrival_time))

    return network_schedule


def connect_stationary_nodes(G: nx.DiGraph) -> None:
    """Connects all nodes to the next node in time at the same station.
    """
    
    nodes_per_station = defaultdict(list)

    for node in G.nodes:
        nodes_per_station[node[0]].append(node)

    for station in nodes_per_station.keys():
        nodes_per_station[station].sort()

        for index, stop in enumerate(nodes_per_station[station][:-1]):
            G.add_edge(stop, nodes_per_station[station][index + 1], edge_attr=(0, 0))


def graph_from_schedule(df: pd.DataFrame) -> nx.DiGraph:
    """Creates a digraph from the schedule.
    """
    
    schedule_network = create_network_schedule(df)

    G = nx.from_pandas_edgelist(
        df=schedule_network, 
        source='start', 
        target='end', 
        edge_attr=['first_class', 'second_class'], 
        create_using=nx.DiGraph
        )
    
    connect_stationary_nodes(G)

    return G

In [29]:
df = traintools.read_schedule("datasets/nsdata1.txt")

G = graph_from_schedule(df)

for stop in nx.dag_longest_path(G, weight='second_class'):
    print(stop)


AttributeError: module 'traintools' has no attribute 'read_schedule'

In [6]:
def fill_train(graph: nx.DiGraph,  route: List[Tuple[int, int]], train_type: Tuple[int, int]):
    """Takes a train and fills it with passengers on the route and returns a new graph.
    """
    
    pass